In [ ]:
%run /home/jovyan/work/database_operations/db_operations.ipynb import DataOperations
%run /home/jovyan/work/ETL/data_extraction.ipynb import ExtractionParamaters
%run /home/jovyan/work/src/transform_data_types.ipynb import assign_data_type
%run /home/jovyan/work/src/load_data.ipynb import load_raw_data


In [ ]:
spark = create_spark_session()
data_operations = DataOperations(spark)

In [ ]:
scorers, results, shootouts = load_raw_data()
scorers, results, shootouts = assign_data_type(scorers, results, shootouts)
# scorers.show()
# shootouts.show()
results.show()

In [ ]:
# print(scorers.count())
print(results.count())
# shootouts.count()
results.show()

In [ ]:
scorers.createOrReplaceTempView("scorers")
results.createOrReplaceTempView("results")
shootouts.createOrReplaceTempView("shootouts")
"""
    WITH query_result AS (
        WITH goal_counts AS (                        
            SELECT date, home_team, away_team,
                SUM(CASE WHEN home_team = team THEN 1 ELSE 0 END) AS home_score,
                SUM(CASE WHEN away_team = team then 1 ELSE 0 END) AS away_score
            FROM scorers
            GROUP BY date, home_team, away_team
    )
        SELECT date, home_team, away_team, home_score, away_score,
            CASE
                WHEN home_score > away_score THEN home_team
                WHEN home_score < away_score THEN away_team
                ELSE 'draw'
            END AS winner
        FROM goal_counts
        ORDER BY date ASC
        )
    SELECT s.date, s.home_team, s.away_team, s.home_score, s.away_score, q.date, q.home_score, q.away_score, q.home_team, q.away_team
    FROM results s
    JOIN query_result q ON s.date = q.date AND s.home_score = q.home_score AND s.away_score = q.away_score AND s.home_team = q.home_team AND s.away_team = q.away_team
    ORDER BY s.date ASC
"""
query_result = spark.sql("""
    (WITH goal_counts AS (                        
        SELECT date, home_team, away_team,
            SUM(CASE WHEN home_team = team THEN 1 ELSE 0 END) AS home_score,
            SUM(CASE WHEN away_team = team then 1 ELSE 0 END) AS away_score
        FROM scorers
        GROUP BY date, home_team, away_team
)
    SELECT date, home_team, away_team, home_score, away_score,
        CASE
            WHEN home_score > away_score THEN home_team
            WHEN home_score < away_score THEN away_team
            ELSE 'draw'
        END AS winner
    FROM goal_counts
    ORDER BY date ASC)
""")
query_result.count()


In [ ]:
result = spark.sql("""
SELECT s.date, s.home_team, s.away_team, s.home_score, s.away_score, q.date, q.home_team, q.away_team, q.home_score, q.away_score 
FROM results s
JOIN query_result q ON s.date = q.date AND s.home_score = q.home_score AND s.away_score = q.away_score AND s.home_team = q.home_team AND s.away_team = q.away_team
ORDER BY s.date ASC
""")
result.count()

In [ ]:
result_2 = spark.sql("""
                   WITH results_2 AS (
                     SELECT date, home_team, away_team, home_score, away_score,
            CASE
                WHEN home_score > away_score THEN home_team
                WHEN home_score < away_score THEN away_team
                ELSE 'draw'
            END AS winner
        FROM results
        ORDER BY date ASC
                     )
SELECT s.date, s.home_team, s.away_team, s.home_score, s.away_score, q.date, q.home_team, q.away_team
FROM results_2 s
JOIN shootouts q ON s.date = q.date AND s.home_team = q.home_team AND s.away_team = q.away_team AND s.winner = q.winner
ORDER BY s.date ASC
                   """)
result_2.show()